# Import useful libraries

In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Useful Functions

In [2]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print("                Model Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (R2 ): {r2}")

In [ ]:
def generate_csv(input, y_pred, key='Validation', filename='output/xgboost_regression_predictions.csv'):
    output = input[['DATETIME', 'ENTITY_DESCRIPTION_SHORT']].copy()
    output['y_pred'] = y_pred
    output['KEY'] = [key] * len(y_pred)
    output.to_csv(filename, index=False)
    print(f"Predictions saved to {filename}")

# Import data

## Simple import

In [ ]:
train_route = '../data/waiting_times_train.csv'
validation_route = '../data/waiting_times_X_test_val.csv'
test_route = '../data/waiting_times_X_test_final.csv'
weather_route = '../data/weather_data.csv'

train_file = pd.read_csv(train_route)
validation_file = pd.read_csv(validation_route)
test_file = pd.read_csv(test_route)
weather_file = pd.read_csv(weather_route)

## Visualize data

### Train File

In [5]:
print(train_file.describe(include='all'))
print("----------------------------------------------------------------------")
print(train_file.dtypes)
print("----------------------------------------------------------------------")
train_file.head()

                   DATETIME ENTITY_DESCRIPTION_SHORT  ADJUST_CAPACITY  \
count                 37018                    37018     37018.000000   
unique                17560                        3              NaN   
top     2021-11-30 13:15:00           Flying Coaster              NaN   
freq                      3                    14468              NaN   
mean                    NaN                      NaN       413.738352   
std                     NaN                      NaN       252.389589   
min                     NaN                      NaN         3.400000   
25%                     NaN                      NaN       247.000000   
50%                     NaN                      NaN       247.000000   
75%                     NaN                      NaN       756.000000   
max                     NaN                      NaN       756.000000   

            DOWNTIME  CURRENT_WAIT_TIME  TIME_TO_PARADE_1  TIME_TO_PARADE_2  \
count   37018.000000       37018.000000     

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H
0,2022-02-05 11:45:00,Water Ride,247.00,0,20,NaN,NaN,NaN,30.0
1,2019-02-24 10:45:00,Water Ride,247.00,0,30,375.0,NaN,495.0,25.0
2,2021-07-17 15:45:00,Pirate Ship,280.50,0,35,NaN,NaN,NaN,35.0
3,2022-04-03 19:45:00,Pirate Ship,230.35,0,15,-135.0,NaN,195.0,10.0
4,2021-10-20 10:30:00,Pirate Ship,153.00,0,15,NaN,NaN,NaN,10.0


### Validation file

In [6]:
print(validation_file.describe(include='all'))
print("----------------------------------------------------------------------")
validation_file.head()

                   DATETIME ENTITY_DESCRIPTION_SHORT  ADJUST_CAPACITY  \
count                  2444                     2444      2444.000000   
unique                 1594                        3              NaN   
top     2022-05-02 19:15:00               Water Ride              NaN   
freq                      3                      901              NaN   
mean                    NaN                      NaN       406.083409   
std                     NaN                      NaN       248.516500   
min                     NaN                      NaN        15.300000   
25%                     NaN                      NaN       247.000000   
50%                     NaN                      NaN       247.000000   
75%                     NaN                      NaN       756.000000   
max                     NaN                      NaN       756.000000   

           DOWNTIME  CURRENT_WAIT_TIME  TIME_TO_PARADE_1  TIME_TO_PARADE_2  \
count   2444.000000        2444.000000       

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW
0,2019-11-23 10:45:00,Water Ride,247.0,0,20,375.0,75.0,675.0
1,2022-01-03 16:45:00,Pirate Ship,153.0,0,45,NaN,NaN,NaN
2,2021-12-04 15:30:00,Pirate Ship,255.0,0,40,NaN,NaN,NaN
3,2020-02-05 13:15:00,Water Ride,247.0,0,15,225.0,NaN,345.0
4,2022-05-13 15:15:00,Flying Coaster,756.0,0,35,135.0,NaN,465.0


### Test file

In [7]:
print(test_file.describe(include='all'))
print("----------------------------------------------------------------------")
test_file.head()

                   DATETIME ENTITY_DESCRIPTION_SHORT  ADJUST_CAPACITY  \
count                  2444                     2444      2444.000000   
unique                 1577                        3              NaN   
top     2020-08-11 12:15:00           Flying Coaster              NaN   
freq                      3                      874              NaN   
mean                    NaN                      NaN       404.383721   
std                     NaN                      NaN       249.284457   
min                     NaN                      NaN         5.100000   
25%                     NaN                      NaN       247.000000   
50%                     NaN                      NaN       247.000000   
75%                     NaN                      NaN       756.000000   
max                     NaN                      NaN       756.000000   

           DOWNTIME  CURRENT_WAIT_TIME  TIME_TO_PARADE_1  TIME_TO_PARADE_2  \
count   2444.000000        2444.000000       

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW
0,2022-04-30 12:15:00,Water Ride,247.0,0,45,315.0,NaN,645.0
1,2022-03-12 11:30:00,Water Ride,247.0,0,45,NaN,NaN,NaN
2,2022-07-03 14:15:00,Flying Coaster,756.0,0,30,195.0,NaN,525.0
3,2019-04-16 15:15:00,Flying Coaster,756.0,0,40,135.0,NaN,405.0
4,2019-10-11 15:15:00,Flying Coaster,756.0,0,25,135.0,-165.0,285.0


### Weather file

In [8]:
print(weather_file.describe(include='all'))
print("----------------------------------------------------------------------")
weather_file.head()

                 temp      dew_point     feels_like       pressure  \
count   135933.000000  135933.000000  135933.000000  135933.000000   
unique            NaN            NaN            NaN            NaN   
top               NaN            NaN            NaN            NaN   
freq              NaN            NaN            NaN            NaN   
mean        12.252124       8.219247      10.968768    1017.220105   
std          7.203461       5.373415       8.293017       9.442864   
min         -6.420000     -36.100000     -13.410000     973.000000   
25%          7.000000       4.230000       4.540000    1012.000000   
50%         11.552500       8.350000      10.912500    1018.000000   
75%         17.140000      12.330000      16.875000    1023.000000   
max         41.260000      25.800000      41.000000    1047.000000   

             humidity     wind_speed        rain_1h        snow_1h  \
count   135933.000000  135933.000000  135917.000000  131121.000000   
unique            N

,temp,dew_point,feels_like,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,DATETIME
0,12.170,7.6800,11.3700,1019.0,74.0,3.3000,NaN,NaN,100.00,2018-10-01 00:00:00
1,11.995,7.6075,11.1925,1019.0,74.5,3.2625,NaN,NaN,99.25,2018-10-01 00:15:00
2,11.820,7.5350,11.0150,1019.0,75.0,3.2250,NaN,NaN,98.50,2018-10-01 00:30:00
3,11.645,7.4625,10.8375,1019.0,75.5,3.1875,NaN,NaN,97.75,2018-10-01 00:45:00
4,11.470,7.3900,10.6600,1019.0,76.0,3.1500,NaN,NaN,97.00,2018-10-01 01:00:00


# Models

The data will be cleaned by different ways and after that a linear regression model will be trained for every type of cleaning. Firstly, column types will be modified.

In [9]:
X_train = train_file
X_train['DATETIME'] = pd.to_datetime(X_train['DATETIME'], errors='coerce')
validation_file['DATETIME'] = pd.to_datetime(validation_file['DATETIME'], errors='coerce')
test_file['DATETIME'] = pd.to_datetime(test_file['DATETIME'], errors='coerce')

weather_file['DATETIME'] = pd.to_datetime(weather_file['DATETIME'], errors='coerce')
weather_file = weather_file.fillna(0)


## Train without weather file

### Simple training

In this case, columns which are not numbers will be dropped and a simple regression model will be trained, (this is a very simple first try).

In [10]:
X_train_dropna = X_train.select_dtypes(include=['number']).drop(columns=['WAIT_TIME_IN_2H']).dropna(axis=1)
y_train_dropna = X_train['WAIT_TIME_IN_2H']

model = XGBRegressor()
model.fit(X_train_dropna, y_train_dropna)

y_pred = model.predict(X_train_dropna)

evaluate_model(y_train_dropna, y_pred)

                Model Evaluation Metrics:
Mean Absolute Error (MAE): 8.666102809553447
Root Mean Squared Error (RMSE): 11.32316113393572
R-squared (R2 ): 0.3714281154725194


### Training using one hot encoding for entity_description_short and replacing NaN by constants

In [11]:
X_train_2 = X_train
X_train_2 = pd.get_dummies(X_train_2, columns=['ENTITY_DESCRIPTION_SHORT'], drop_first=True, dtype=int)
X_train_2['year'] = X_train_2['DATETIME'].dt.year
X_train_2['month'] = X_train_2['DATETIME'].dt.month
X_train_2['day'] = X_train_2['DATETIME'].dt.day
X_train_2['hour'] = X_train_2['DATETIME'].dt.hour
X_train_2['minute'] = X_train_2['DATETIME'].dt.minute
X_train_2 = X_train_2.select_dtypes(include=['number']).drop(columns=['WAIT_TIME_IN_2H']).fillna(24*60)
y_train_2 = X_train['WAIT_TIME_IN_2H']



'''optimized_estimators = 0
max_score = 0
for n in range(5, 100, 500):
    model = XGBRegressor()
    scores = cross_val_score(model, X_train_2, y_train_2, scoring='r2', cv=5)
    score_mean = scores.mean()
    if score_mean > max_score:
        max_score = score_mean
        optimized_estimators = n


print(optimized_estimators)'''

model = XGBRegressor()
scores = cross_val_score(model, X_train_2, y_train_2, scoring='r2', cv=5)
print(scores.mean(), scores.std())
model.fit(X_train_2, y_train_2)

y_pred = model.predict(X_train_2)

evaluate_model(y_train_2, y_pred)

0.7068593063298134 0.007991666139650818
                Model Evaluation Metrics:
Mean Absolute Error (MAE): 4.949656653182416
Root Mean Squared Error (RMSE): 6.657091070758497
R-squared (R2 ): 0.7827355988394628


# Train with weather file

In [13]:
X_train_2 = pd.merge(X_train, weather_file, on=['DATETIME'], how='left')
X_train_2 = pd.get_dummies(X_train_2, columns=['ENTITY_DESCRIPTION_SHORT'], drop_first=True, dtype=int)
X_train_2['year'] = X_train_2['DATETIME'].dt.year
X_train_2['month'] = X_train_2['DATETIME'].dt.month
X_train_2['day'] = X_train_2['DATETIME'].dt.day
X_train_2['hour'] = X_train_2['DATETIME'].dt.hour
X_train_2['minute'] = X_train_2['DATETIME'].dt.minute
X_train_2 = X_train_2.select_dtypes(include=['number']).drop(columns=['WAIT_TIME_IN_2H']).fillna(24*60)
X_train_2 = X_train_2.drop(columns=['dew_point', 'humidity', 'clouds_all', 'temp'])
print(X_train_2.head())
y_train_2 = X_train['WAIT_TIME_IN_2H']

optimized_estimators = 100
max_score = 0
for n in range(500, 1001, 100):
    model = XGBRegressor(n_estimators=n)
    scores = cross_val_score(model, X_train_2, y_train_2, scoring='r2', cv=5)
    score_mean = scores.mean()
    if score_mean > max_score:
        max_score = score_mean
        optimized_estimators = n


print(optimized_estimators)

model = XGBRegressor(n_estimators=optimized_estimators)
scores = cross_val_score(model, X_train_2, y_train_2, scoring='neg_root_mean_squared_error', cv=5)
print(scores.mean(), scores.std())
model.fit(X_train_2, y_train_2)

y_pred = model.predict(X_train_2)
evaluate_model(y_train_2, y_pred)

X_val = pd.merge(validation_file, weather_file, on=['DATETIME'], how='left')
X_val = pd.get_dummies(X_val, columns=['ENTITY_DESCRIPTION_SHORT'], drop_first=True, dtype=int)
X_val['year'] = X_val['DATETIME'].dt.year
X_val['month'] = X_val['DATETIME'].dt.month
X_val['day'] = X_val['DATETIME'].dt.day
X_val['hour'] = X_val['DATETIME'].dt.hour
X_val['minute'] = X_val['DATETIME'].dt.minute
X_val = X_val.select_dtypes(include=['number']).fillna(24*60)
X_val = X_val.drop(columns=['dew_point', 'humidity', 'clouds_all', 'temp'])
y_pred_final = model.predict(X_val)

generate_csv(validation_file, y_pred_final)

   ADJUST_CAPACITY  DOWNTIME  CURRENT_WAIT_TIME  TIME_TO_PARADE_1  \
0           247.00         0                 20            1440.0   
1           247.00         0                 30             375.0   
2           280.50         0                 35            1440.0   
3           230.35         0                 15            -135.0   
4           153.00         0                 15            1440.0   

   TIME_TO_PARADE_2  TIME_TO_NIGHT_SHOW  feels_like  pressure  wind_speed  \
0            1440.0              1440.0       3.630   1026.75      3.1500   
1            1440.0               495.0       6.010   1035.75      3.0025   
2            1440.0              1440.0      22.040   1025.00      4.5325   
3            1440.0               195.0       5.275   1023.00      2.3425   
4            1440.0              1440.0      15.020   1010.50      6.8850   

    rain_1h   snow_1h  ENTITY_DESCRIPTION_SHORT_Pirate Ship  \
0  0.383750  0.139065                                     0